In [1]:
import sys
import os

sys.path.append(os.getcwd()[:-9])
from src.ops.operator import Operator
from src.ops.elemental import ElementalOperator
from src.ops.quarks import Quark
from src.ops.indexed import IndexedObject
from src.wick.contract import contract

In [2]:
q0=Quark(True,'u','s_0','c_0','t_3','x_3')
g0=IndexedObject('\\gamma_5',['s_0','s_1'])
q1=Quark(False,'d','s_1','c_0','t_3','x_3')

annihilate=Operator([ElementalOperator(+1,[g0],[q0,q1])])

#TODO add charge factors qu and qd

elementals=[]
for pi2 in [['u',+2/3],['d',-1/3]]:                              # uu + dd
    for pi1 in [['u',+2/3],['d',-1/3]]:                          # uu + dd 
        for dirpi2 in [-1,+1]:                               # eq 17 of notes
            for dirpi1 in [-1,+1]:
                q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2')
                g1=IndexedObject('(1-\\gamma_{\mu})',['s_2','s_3'])
                g2=IndexedObject('U_{\mu}(x_2)',['c_1','c_2'])
                q3=Quark(False,pi2[0],'s_3','c_2','t_2','x_2+a\hat{\mu}')
                q4=Quark(True,pi1[0],'s_4','c_3','t_1','x_1')
                g3=IndexedObject('(1-\\gamma_{\\nu})',['s_4','s_5'])
                g4=IndexedObject('U_{\\nu}(x_1)',['c_3','c_4'])
                q5=Quark(False,pi1[0],'s_5','c_4','t_1','x_1+a\hat{\\nu}')
                if(dirpi1==-1):
                    q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2')
                    g1=IndexedObject('(1-\\gamma_{\mu})',['s_2','s_3'])
                    g2=IndexedObject('U_{\mu}(x_2)',['c_1','c_2'])
                    q3=Quark(False,pi2[0],'s_3','c_2','t_2','x_2+a\hat{\mu}')
                elif(dirpi1==+1):
                    q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2+a\hat{\mu}')
                    g1=IndexedObject('(1+\\gamma_{\mu})',['s_2','s_3'])
                    g2=IndexedObject('U_{\mu}^{\dagger}(x_2)',['c_1','c_2'])
                    q3=Quark(False,pi2[0],'s_3','c_2','t_2','x_2')
    
                if(dirpi2==-1):
                    q4=Quark(True,pi1[0],'s_4','c_3','t_1','x_1')
                    g3=IndexedObject('(1-\\gamma_{\\nu})',['s_4','s_5'])
                    g4=IndexedObject('U_{\\nu}(x_1)',['c_3','c_4'])
                    q5=Quark(False,pi1[0],'s_5','c_4','t_1','x_1+a\hat{\\nu}')
                    
                elif(dirpi2==+1):
                    q4=Quark(True,pi1[0],'s_4','c_3','t_1','x_1+a\hat{\\nu}')
                    g3=IndexedObject('(1+\\gamma_{\\nu})',['s_4','s_5'])
                    g4=IndexedObject('U_{\\nu}^{\dagger}(x_1)',['c_3','c_4'])
                    q5=Quark(False,pi1[0],'s_5','c_4','t_1','x_1')
        
                # for printing out the insertion cleanly, just comment out this pion
                # and uncomment out the block about insertion.
                q6=Quark(False,'u','s_6','c_3','t_0','x_0')
                g5=IndexedObject('\\gamma_5',['s_6','s_7'])
                q7=Quark(True,'d','s_7','c_3','t_0','x_0')
                
                coef=pi1[1]*pi2[1]*dirpi1*dirpi2
                elementals.append(ElementalOperator(coef,[g1,g2,g3,g4,g5],[q2,q3,q4,q5,q6,q7]))
                #elementals.append(ElementalOperator(coef,[g1,g2,g3,g4],[q2,q3,q4,q5]))
        
insertion=Operator(elementals)
s=''
for e in insertion.elementals:
    s+=str(e)+'\\\\'
print(s.replace('*',''))

creation=Operator(elementals)

0.4444444444444444(1-\gamma_{\mu})_{s_2 s_3}U_{\mu}(x_2)_{c_1 c_2}(1-\gamma_{\nu})_{s_4 s_5}U_{\nu}(x_1)_{c_3 c_4}\gamma_5_{s_6 s_7}\bar{u}_{s_2 c_1}(x_2, t_2)u_{s_3 c_2}(x_2+a\hat{\mu}, t_2)\bar{u}_{s_4 c_3}(x_1, t_1)u_{s_5 c_4}(x_1+a\hat{\nu}, t_1)u_{s_6 c_3}(x_0, t_0)\bar{d}_{s_7 c_3}(x_0, t_0)\\-0.4444444444444444(1+\gamma_{\mu})_{s_2 s_3}U_{\mu}^{\dagger}(x_2)_{c_1 c_2}(1-\gamma_{\nu})_{s_4 s_5}U_{\nu}(x_1)_{c_3 c_4}\gamma_5_{s_6 s_7}\bar{u}_{s_2 c_1}(x_2+a\hat{\mu}, t_2)u_{s_3 c_2}(x_2, t_2)\bar{u}_{s_4 c_3}(x_1, t_1)u_{s_5 c_4}(x_1+a\hat{\nu}, t_1)u_{s_6 c_3}(x_0, t_0)\bar{d}_{s_7 c_3}(x_0, t_0)\\-0.4444444444444444(1-\gamma_{\mu})_{s_2 s_3}U_{\mu}(x_2)_{c_1 c_2}(1+\gamma_{\nu})_{s_4 s_5}U_{\nu}^{\dagger}(x_1)_{c_3 c_4}\gamma_5_{s_6 s_7}\bar{u}_{s_2 c_1}(x_2, t_2)u_{s_3 c_2}(x_2+a\hat{\mu}, t_2)\bar{u}_{s_4 c_3}(x_1+a\hat{\nu}, t_1)u_{s_5 c_4}(x_1, t_1)u_{s_6 c_3}(x_0, t_0)\bar{d}_{s_7 c_3}(x_0, t_0)\\0.4444444444444444(1+\gamma_{\mu})_{s_2 s_3}U_{\mu}^{\dagger}(x_2)_{c_1 c_2}(1

In [3]:
res=contract(annihilate,creation)
print(len(res))
res[0].coef

80


-0.4444444444444444

In [7]:
import copy

def latexStr(gamma):
    tmp=gamma.name.split('_')
    if(tmp[0]=='(1-\\gamma)') or (tmp[0]=='(1+\\gamma)'):
        return tmp[0] + '_{' + tmp[1][:-1] + '_{' + gamma.indices[0] + gamma.indices[1] + '}}'
    elif tmp[0][0]=='U':
        return '('+gamma.name+')'+ '_{{' + gamma.indices[0] + gamma.indices[1] + '}}'
    elif tmp[0]=='\\gamma':
        return tmp[0] + '_{' + tmp[1] + '_{' + gamma.indices[0] + gamma.indices[1] + '}}'
    else:
        return str(gamma)
  
def platexStr(p):
    return p.name + '(' + p.xi + ',' + p.ti + '\mid ' + p.xf + ',' + p.tf + ')' + '_{\\substack{' + p.left_indices.s + '\\\\' + p.left_indices.c + '}' +  '\\substack{' + p.right_indices.s + '\\\\' + p.right_indices.c + '}}'
    
total=copy.deepcopy(res)

print(len(total))

#for d in total:
    #for p in d.props:
        #p.name = 'S'
        
for i in range(len(total)-1,-1,-1):
    for j in range(len(total)-1,-1,-1):
        if(i!=j):
            if((total[i].commuting==total[j].commuting) and (total[i].props==total[j].props)):
                total[i].coef+=total[j].coef
                total.pop(j)
                break
for i in range(len(total)-1,-1,-1):
    if(total[i].coef==0):
        total.pop(i)
        
print(len(total))

80
80


In [8]:
print('\\beqs')
for i,d in enumerate(total):
    #if(i==1 or i==6):
        tmp='d_{'+str(i)+'}&='+str(int(d.coef*9)) + '/9 '
        for p in d.props:
            tmp+=platexStr(p)
        tmp+='\\\\&'
        for c in d.commuting:
            tmp+=latexStr(c)
        print(tmp+'\\\\')
        if(i%12==0 and i!=0):
            print('\\eeqs')
            print('\\beqs')
print('\\eeqs')

\beqs
d_{0}&=-4/9 prop^u(x_0,t_0\mid x_3,t_3)_{\substack{s_6\\c_3}\substack{s_0\\c_0}}prop^u(x_1+a\hat{\nu},t_1\mid x_2,t_2)_{\substack{s_5\\c_4}\substack{s_2\\c_1}}prop^u(x_2+a\hat{\mu},t_2\mid x_1,t_1)_{\substack{s_3\\c_2}\substack{s_4\\c_3}}prop^d(x_3,t_3\mid x_0,t_0)_{\substack{s_1\\c_0}\substack{s_7\\c_3}}\\&(1-\gamma_{\mu})_{s_2 s_3}(1-\gamma_{\nu})_{s_4 s_5}(U_{\mu}(x_2))_{{c_1c_2}}(U_{\nu}(x_1))_{{c_3c_4}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\\
d_{1}&=4/9 prop^u(x_1+a\hat{\nu},t_1\mid x_3,t_3)_{\substack{s_5\\c_4}\substack{s_0\\c_0}}prop^u(x_0,t_0\mid x_2,t_2)_{\substack{s_6\\c_3}\substack{s_2\\c_1}}prop^u(x_2+a\hat{\mu},t_2\mid x_1,t_1)_{\substack{s_3\\c_2}\substack{s_4\\c_3}}prop^d(x_3,t_3\mid x_0,t_0)_{\substack{s_1\\c_0}\substack{s_7\\c_3}}\\&(1-\gamma_{\mu})_{s_2 s_3}(1-\gamma_{\nu})_{s_4 s_5}(U_{\mu}(x_2))_{{c_1c_2}}(U_{\nu}(x_1))_{{c_3c_4}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\\
d_{2}&=-4/9 prop^u(x_2+a\hat{\mu},t_2\mid x_3,t_3)_{\substack{s_3\\c_2}\substack{s_0\\c_0}}

In [9]:
diagrams = {}

for i,d in enumerate(total):
    top = []
    d_str='d_' + str(i) + ':  '
    for p in d.props:
        d_str+=p.tf + '->' + p.ti + '   '
        top.append([p.tf.split('_')[1],p.ti.split('_')[1]])
    diagrams[i]=top
    print(d_str)

d_0:  t_3->t_0   t_2->t_1   t_1->t_2   t_0->t_3   
d_1:  t_3->t_1   t_2->t_0   t_1->t_2   t_0->t_3   
d_2:  t_3->t_2   t_2->t_0   t_1->t_1   t_0->t_3   
d_3:  t_3->t_0   t_2->t_2   t_1->t_1   t_0->t_3   
d_4:  t_3->t_1   t_2->t_2   t_1->t_0   t_0->t_3   
d_5:  t_3->t_2   t_2->t_1   t_1->t_0   t_0->t_3   
d_6:  t_3->t_0   t_2->t_1   t_1->t_2   t_0->t_3   
d_7:  t_3->t_1   t_2->t_0   t_1->t_2   t_0->t_3   
d_8:  t_3->t_2   t_2->t_0   t_1->t_1   t_0->t_3   
d_9:  t_3->t_0   t_2->t_2   t_1->t_1   t_0->t_3   
d_10:  t_3->t_1   t_2->t_2   t_1->t_0   t_0->t_3   
d_11:  t_3->t_2   t_2->t_1   t_1->t_0   t_0->t_3   
d_12:  t_3->t_0   t_2->t_1   t_1->t_2   t_0->t_3   
d_13:  t_3->t_1   t_2->t_0   t_1->t_2   t_0->t_3   
d_14:  t_3->t_2   t_2->t_0   t_1->t_1   t_0->t_3   
d_15:  t_3->t_0   t_2->t_2   t_1->t_1   t_0->t_3   
d_16:  t_3->t_1   t_2->t_2   t_1->t_0   t_0->t_3   
d_17:  t_3->t_2   t_2->t_1   t_1->t_0   t_0->t_3   
d_18:  t_3->t_0   t_2->t_1   t_1->t_2   t_0->t_3   
d_19:  t_3->t_1   t_2-

In [10]:
refTopologies={}
refTopologies['A']=[['1','0'],['0','2'],['2','3'],['3','1']]
refTopologies['A-bwd']=[['0','1'],['1','3'],['3','2'],['2','0']]
refTopologies['B']=[['1','0'],['0','3'],['3','2'],['2','1']]
refTopologies['B-bwd']=[['0','1'],['1','2'],['2','3'],['3','0']]
refTopologies['C']=[['2','0'],['0','3'],['3','1'],['1','2']]
refTopologies['C-bwd']=[['0','2'],['2','1'],['1','3'],['3','0']]
refTopologies['D']=[['0','3'],['3','0'],['2','1'],['1','2']]
refTopologies['El']=[['2','2'],['3','1'],['1','0'],['0','3']]
refTopologies['El-bwd']=[['2','2'],['1','3'],['3','0'],['0','1']]
refTopologies['Er']=[['1','1'],['3','2'],['2','0'],['0','3']]
refTopologies['Er-bwd']=[['1','1'],['2','3'],['3','0'],['0','2']]
refTopologies['F']=[['0','3'],['3','0'],['1','1'],['2','2']]

In [11]:
def swap(list, i, j):
    list[i], list[j] = list[j], list[i]
    return list

for idx,d in diagrams.items():
    for p in range(len(d)):
        if d[p][1] != d[(p+1)%len(d)][0]:
            for pp in range(p,len(d)):
                if d[p][1] == d[pp][0]:
                    swap(d,(p+1)%len(d),pp)

In [12]:
from src.wick.utilities import permutations

def cyclic_permutations(list):
    return [[list[i-j] for i in range(len(list))] for j in range(len(list))]

print('\\begin{tabular}{c | c}')
print('d & Diagrams\\\\')
print('\\hline')
for idx,d in diagrams.items():
    for label, top in refTopologies.items():
        if d in permutations(top):
            print('{} & {}\\\\'.format(idx,label[0]))
print('\\end{tabular}')

\begin{tabular}{c | c}
d & Diagrams\\
\hline
0 & D\\
1 & C\\
2 & E\\
3 & F\\
4 & E\\
5 & B\\
6 & D\\
7 & C\\
8 & E\\
9 & F\\
10 & E\\
11 & B\\
12 & D\\
13 & C\\
14 & E\\
15 & F\\
16 & E\\
17 & B\\
18 & D\\
19 & C\\
20 & E\\
21 & F\\
22 & E\\
23 & B\\
24 & E\\
25 & A\\
26 & E\\
27 & F\\
28 & E\\
29 & A\\
30 & E\\
31 & F\\
32 & E\\
33 & A\\
34 & E\\
35 & F\\
36 & E\\
37 & A\\
38 & E\\
39 & F\\
40 & A\\
41 & E\\
42 & F\\
43 & E\\
44 & A\\
45 & E\\
46 & F\\
47 & E\\
48 & A\\
49 & E\\
50 & F\\
51 & E\\
52 & A\\
53 & E\\
54 & F\\
55 & E\\
56 & E\\
57 & B\\
58 & E\\
59 & C\\
60 & D\\
61 & F\\
62 & E\\
63 & B\\
64 & E\\
65 & C\\
66 & D\\
67 & F\\
68 & E\\
69 & B\\
70 & E\\
71 & C\\
72 & D\\
73 & F\\
74 & E\\
75 & B\\
76 & E\\
77 & C\\
78 & D\\
79 & F\\
\end{tabular}
